在Python中，有一种非常简单的方法可以判断一个元素是否在列表中。就是`in`运算符

In [1]:
15 in [1,2,3,4,5]

False

### 顺序搜索
当数据项存储在列表等集合中时，我们说它们具有线性或顺序关系。每个数据项都存储在相对于其他数据项的位置上。在Python列表中，这些相对位置是各个项的索引值。由于这些索引值是有序的，我们可以按顺序访问它们。这个过程产生了我们的第一种搜索技术——**顺序搜索**。
![](/img/python/sequential-search.png)
- **无序列表顺序搜索比较**
| Case 情况               | Best Case 最佳情况 | Worst Case 最坏情况 | Average Case 平均情况 |
|-------------------------|--------------------|--------------------|----------------------|
| 项目存在 | 1                  | n                  | n/2                  |
| 不存在 | n                | n                  | n                    |
- **有序列表顺序搜索比较**
| Case 情况               | Best Case 最佳情况 | Worst Case 最坏情况 | Average Case 平均情况 |
|-------------------------|--------------------|--------------------|----------------------|
| 项目存在 | 1                  | n                  | n/2                  |
| 不存在 | 1                | n                  | n/2                  |

这个比较简单，无序列表搜索纯遍历，有序列表顺序搜索早停就行
### 二分查找
![](/img/python/binary-search.png)


In [2]:
def binarySearch(alist, item):
    first = 0
    last = len(alist) - 1
    found = False

    while first <= last and not found:
        midpoint = (first + last) // 2 #  // 是整除运算符，返回商的整数部分
        if alist[midpoint] == item:
            found = True
        else:
            if item < alist[midpoint]:
                last = midpoint - 1 # -1 是因为 midpoint 已经被检查过了，所以不需要再检查
            else:
                first = midpoint + 1 # +1 是因为 midpoint 已经被检查过了，所以不需要再检查
    return found

testlist = [0, 1, 2, 8, 13, 17, 19, 32, 42,]
print(binarySearch(testlist, 3))
print(binarySearch(testlist, 13))

False
True


该算法是分治策略的一个绝佳例子。分治意味着我们将问题分解成更小的部分，以某种方式解决这些小部分，然后重新组合整个问题以得出结果。
当我们对一个列表执行二分查找时，首先会检查中间的元素。如果我们要查找的元素小于中间元素，我们只需对原始列表的左半部分执行二分查找。
同样，如果该元素更大，我们可以对右半部分执行二分查找。无论哪种情况，这都是对二分查找函数的递归调用，传递的是一个更小的列表(切片)

In [3]:
def binarySearch(blist, item):
    if len(blist) == 0:
        return False
    else:
        midpoint = len(blist) // 2
        if blist[midpoint] == item:
            return True
        else:
            if item < blist[midpoint]:
                return binarySearch(blist[:midpoint], item)
            else:
                return binarySearch(blist[midpoint+1:], item) # 细节左闭右开
                
testlist = [0, 1, 2, 8, 13, 17, 19, 32, 42,]
print(binarySearch(testlist, 3))
print(binarySearch(testlist, 13))

False
True


| Comparisons 比较次数 | Approximate Number of Items Left 剩余物品的大致数量 |
|----------------------|----------------------------------------------------|
| 1                    | n/2                                                |
| 2                    | n/4                                                |
| 3                    | n/8                                                |
| …                    | …                                                  |
| i                    | n/(2^i)                                            |

- 到达只剩 1 个元素的比较次数 i 满足：2^i = n；
- 求解 i 得：i = log₂n；
- 二分查找的时间复杂度是：O(log n)。

> BTW，切片运算符会破坏对数时间，比如切片操作 alist[a:b] 时间复杂度是 O(k)（k 是切片长度），相当于创建一个新的长度为 k 的列表，这会让二分查找「不再是严格的 O (log n)」；
>
> 更优解是不要切片，而是给函数传「列表 + 起始索引 + 结束索引」（比如 binarySearch(alist, left, right, item)），通过计算索引来限定查找范围，避免切片的额外开销。

二分查找虽然比顺序查找快，但不是所有场景都该用：
- 前提：二分查找要求列表是有序的（排序本身有成本）；
- 小列表（n 小）：排序的额外成本 > 二分查找的收益，不如直接用顺序查找；
- 大列表：
  - 若「排序 1 次 + 多次查找」：排序成本分摊后，二分查找很划算；
  - 若「只查 1 次」：排序的成本太高（排序是 O (n log n)），不如直接顺序查找（O (n)）。

二分查找是否划算，取决于「排序成本」和「查找次数」的权衡 —— 仅当「排序 1 次、多次查找」时，二分查找的优势才明显。

### 哈希
前面学的二分查找依赖「列表有序」，能做到 O (logn) 的对数时间搜索；而哈希表想更进一步 ——通过 “精准定位元素的存储位置”，实现仅需 1 次比较的 O (1) 常数时间搜索。

哈希表的基本单元是「**槽（slot）**」：每个槽有唯一编号（从 0 开始），初始是空的（Python 用 None 表示）；
- 示例：大小为 11 的哈希表 = 11 个槽（编号 0~10），可以用 Python 列表实现（初始全为 None）；
  
**负载因子**：已占用槽数 / 总槽数（示例中 6 个元素占 11 个槽，负载因子 = 6/11≈0.545）。
![](/img/python/hash-table-eg.png)
![](/img/python/hash-table-eg1.png)

哈希表能快速查找的核心是「**哈希函数（Hash Function）**」—— 它的作用是：把任意元素（数字、字符串等）转换成 0~m-1 的整数（槽位编号）。
- 最常用的是余数法：大小为 11 的哈希表，哈希函数 h(item) = item%11（对 11 取余）。
  - 54 % 11 = 10 → 54 放在槽 10；
  - 77 % 11 = 0 → 77 放在槽 0；
  - 44 % 11 = 0 → 和 77 的槽位重复（这就是「冲突」）。
设计目标：冲突少、计算快、元素分布均匀（三个核心要求），除了基础的余数法，还有这些扩展方法：

| 哈希方法   | 核心逻辑                                                                 | 示例（表大小 = 11）                              |
|------------|--------------------------------------------------------------------------|-------------------------------------------------|
| 折叠法     | 把元素拆成等长片段→相加→取余                                             | 电话号码 436-555-4601 拆成 43+65+55+46+01=210 → 210%11=1 |
| 平方取中法 | 元素平方→取中间几位数字→取余                                             | 44²=1936 → 取中间 93 → 93%11=5                  |
| 字符串哈希 | 字符转 ASCII 值相加→取余（变位词会冲突，可加 “位置权重” 解决）| "cat"=99+97+116=312 → 312%11=4                  |

|  |  |
| --- | --- |
| ![](/img/python/hash-table-ascii1.png) | ![](/img/python/hash-table-ascii2.png) |
> 哈希的核心是 “快”，如果哈希函数设计得太复杂（比如拆片段 + 反转 + 多层计算），计算哈希值的时间比顺序查找 / 二分查找还久，那哈希表的优势就完全没了 —— 这是设计哈希函数的底线

理想状态是，每个元素通过哈希函数映射到唯一槽位（这种哈希函数叫「完美哈希函数」）

完美哈希函数只有 “元素固定且数量极少” 时能实现（比如只存 25 个学生的信息）；

如果元素范围大（比如 9 位社保号），要做完美哈希需要 10 亿个槽位，完全浪费内存；

因此实际中「冲突（Collision）」不可避免（多个元素映射到同一个槽位）

#### 哈希冲突解决
系统的处理「多个元素哈希到同一个槽位」核心思路分两种：
- 开放寻址（Open Addressing）：冲突元素「换个空槽位坐」
  - 线性探测（Linear Probing）：从冲突槽位开始，依次检查下一个槽位（+1），直到找到空槽位或遍历完所有槽位。
  - 二次探测（Quadratic Probing）：探测间隔不是 +1，而是 +1^2、+2^2、+3^2 等。
- 链地址法（Chaining）：冲突元素「挤在同一个槽位坐」，每个槽位维护一个链表，冲突元素都链在链表后面。

开放定址法核心逻辑是从冲突的原槽位开始，按特定规则遍历哈希表，找到第一个空槽位存放元素；同理查找时必须用相同规则，否则会找不到元素
![](/img/python/hash-linear-prob.png)
其中**线性探测**又是最基础的开放定址法
- 44 哈希到槽 0（已被 77 占用）→ 查槽 1（空），放入 44；
- 55 哈希到槽 0→ 槽 0/1 都满→ 查槽 2（空），放入 55；
- 20 哈希到槽 9（已被 31 占用）→ 依次查槽 10、0、1、2→ 槽 3 空，放入 20；
找20的时候 先查槽 9（不是）→ 按线性探测规则往后找，直到找到 20 或空槽（空槽说明不存在）
![](/img/python/hash-linear-prob-cluster.png)
其致命缺点是**聚集（Clustering）** —— 一个槽位冲突会导致周围槽位都被占（比如槽 0 的冲突导致槽 1、2、3 都被占），形成 “集群”，严重降低后续插入 / 查找效率

为了解决聚集问题，无非就是增大步长，然后根据步长是线性增长还是指数增长又分为两种，这里先看**再哈希（Rehashing）**
- 原理：不再逐个找，而是按固定步长跳着找（比如步长 3），避免扎堆；
- 示例：步长 3→ 冲突后找「原槽位 + 3→+6→+9…」（循环遍历）；
- 关键注意：
  - 步长必须保证能遍历整个哈希表（否则部分槽位永远用不上）；
  - 哈希表大小建议选质数（比如示例中的 11）—— 若表大小是 10（合数）、步长 2，只能查偶数槽位，奇数槽位永远空着；
  - 通用公式：再哈希值 = (原哈希值 + 步长×i) % 表大小（i 是探测次数）。
![](/img/python/hash-linear-prob-plus.png)

**二次探测（Quadratic Probing）**则是动态步长的再哈希
- 原理：步长不是固定值，而是平方数递增（1、4、9、16…）→ 原哈希值 h→ 依次查 h+1、h+4、h+9…；
- 优势：聚集问题比线性探测轻得多（跳着找，不会扎堆在原槽位周围）；
- 公式：再哈希值 = (h + i²) % 表大小（i 是探测次数，1、2、3…）。
![](/img/python/hash-quadratic-prob.png)

工业界最常用的冲突解决方法（比如 Python 字典的底层实现）则是使用的**链地址法（Chaining）**
![](/img/python/hash-chaining.png)
- 原理：每个槽位不存单个元素，而是存一个链表 / 集合；冲突元素直接加到原槽位的链表中；
- 示例：槽 0 原本存 77→ 44、55 哈希到槽 0→ 槽 0 的链表变成 [77, 44, 55]；
- 查找逻辑：先哈希到目标槽位→ 遍历该槽位的链表，判断元素是否存在；
- 核心优势：
  - 完全避免 “聚集” 问题（冲突元素只在本槽位的链表中，不影响其他槽位）；
  - 平均每个链表的元素数量少，查找效率仍接近 O (1)；
- 小缺点：每个槽位要维护链表，略增加一点内存开销（但性价比极高）。

### 表（Map）
咱都知道 Python 里有一个`Dict`（Dictionary）的数据结构，实际上就是 Python 对 Map 这个抽象概念的具体实现。我们的 C 语言里没有内置的 Map 或 Dict，C 程序员需要自己用哈希表 / 链表等手动实现键值对映射的功能，所以假装我们是一个写 C 的，用 Python 感受一下吧：

Map 抽象数据类型的定义如下。该结构是键与数据值之间关联的无序集合。映射中的键都是唯一的，因此键和值之间存在一一对应的关系。其操作如下。
- Map() 创建一个新的空映射。它返回一个空的映射集合。
- put(key,val) 向映射中添加新的键值对。如果该键已在映射中，则用新值替换旧值。
- get(key) 给定一个键，返回映射中存储的值，否则返回None。
- del 使用del map[key]形式的语句从映射中删除键值对。
- len() 返回映射中存储的键值对数量。
- in 对于 key in map 形式的语句，如果给定的键在映射中，则返回 True，否则返回 False。

字典的一大优势在于，给定一个键，我们可以非常快速地查找相关联的数据值。为了提供这种快速查找能力，我们需要一种支持高效搜索的实现方式。我们可以使用列表进行顺序搜索或二分搜索，但使用上述的哈希表会更好，因为在哈希表中查找一个元素的性能可以接近O(1)

In [7]:
class HashTable:
    def __init__(self):
        self.size = 11 # 质数这样冲突解决算法才能尽可能高效
        self.slots = [None] * self.size # \* 表示创建一个长度为 self.size 的列表，每个元素都是 None
        self.data = [None] * self.size
    def hashfunction(self, key, size):
        return key % size # 简单的取余法
    def rehash(self, oldhash, size):
        return (oldhash + 1) % size # 线性探测+1，循环遍历

    # 核心，添加/修改键值对
    def put(self, key, data):
        # 首先计算初始哈希值，看看放哪个slot
        hashvalue = self.hashfunction(key, len(self.slots))

        # 情况1：槽位为空 → 直接存key和value
        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        # 情况2：key是一样的 → 替换value（更新）
        elif self.slots[hashvalue] == key:
            self.data[hashvalue] = data # 替换更新旧值
        # 情况3：槽位被其他key占用（冲突）→ 线性探测找空槽/相同key的槽
        else:
            nextslot = self.rehash(hashvalue, len(self.slots))
            # 循环探测：直到找到空槽 或 找到相同key 或 遍历完
            while self.slots[nextslot] != None and self.slots[nextslot] != key:
                nextslot = self.rehash(nextslot, len(self.slots))
            # 情况3.1：找到空槽 → 存key和value
            if self.slots[nextslot] == None:
                self.slots[nextslot] = key
                self.data[nextslot] = data
            # 情况3.2：找到相同key的槽 → 替换value（更新）
            else:
                self.data[nextslot] = data # 替换更新旧值

    def get(self, key):
        startslot = self.hashfunction(key, len(self.slots))
        data = None
        stop = None   # 是否遍历完所有槽位
        found = False
        position = startslot

        while self.slots[position] != None and not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position = self.rehash(position, len(self.slots))
                # 回到初始槽 → 遍历完所有可能，停止
                if position == startslot:
                    stop = True
        return data

    # 重载overloading
    # 使Hashtable可以使用 [] 来访问和设置键值对
    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)
    

- 不重载这两个方法时:
```python
H = HashTable()
# 存值：必须调用 put 方法
H.put(54, "cat")
# 取值：必须调用 get 方法
print(H.get(54))  # 输出 "cat"
```
- 重载后，可以像用字典一样用 HashTable：
```python
H = HashTable()
# 存值：可以直接用 [] 赋值
H[54] = "cat"
# 取值：可以直接用 [] 取值
print(H[54])  # 输出 "cat"
```

In [8]:
# 创建哈希表
H = HashTable()
# 存键值对（用[]，因为重载了__setitem__）
H[54] = "cat"
H[26] = "dog"
H[93] = "lion"
H[17] = "tiger"
H[77] = "bird"
H[31] = "cow"
H[44] = "goat"
H[55] = "pig"
H[20] = "chicken"

# 查看存key的slots列表（线性探测后的结果）
print(H.slots)  # [77, 44, 55, 20, 26, 93, 17, None, None, 31, 54]
# 查看存value的data列表（和slots索引一一对应）
print(H.data)   # ['bird', 'goat', 'pig', 'chicken', 'dog', 'lion', 'tiger', None, None, 'cow', 'cat']

# 访问值
print(H[20])    # chicken
print(H[17])    # tiger

# 修改值
H[20] = "duck"
print(H[20])    # duck

# 访问不存在的key → 返回None
print(H[99])    # None

[77, 44, 55, 20, 26, 93, 17, None, None, 31, 54]
['bird', 'goat', 'pig', 'chicken', 'dog', 'lion', 'tiger', None, None, 'cow', 'cat']
chicken
tiger
duck
None


哈希表的性能不是绝对的 O (1)，而是和「负载因子 λ」强相关
- λ 越小（表越空）：冲突越少，查找越接近 O (1)；
- λ 越大（表越满）：冲突越多，线性探测的「聚集」越严重，链地址法的链表越长，查找效率越低。

| 冲突解决方法         | 成功搜索的平均比较次数 | 失败搜索的平均比较次数 |
| -------------------- | ---------------------- | ---------------------- |
| 线性探测（开放定址） | (1 + 1/(1-λ)) / 2      | (1 + 1/(1-λ)²) / 2     |
| 链地址法             | 1 + λ/2                | λ                      |

- 线性探测：失败搜索的性能下降远快于成功搜索（比如 λ=0.8 时，失败搜索平均要≈3.25 次比较，成功搜索≈1.25 次）；
- 链地址法：性能更稳定，即使 λ 增大，比较次数增长也更平缓（比如 λ=0.8 时，成功搜索≈1.4 次，失败搜索≈0.8 次）。

> 实际应用中会「扩容哈希表」（比如 λ 超过 0.7 就扩容），让 λ 保持在 0.5~0.7 之间，保证接近 O (1) 的效率